In [1]:
import numpy as np                                  # linear algebra
import pandas as pd                                 # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import model_selection


from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle

Using TensorFlow backend.


In [3]:
data = pd.read_csv('jobclassification.csv')

In [4]:
data.head()

,ID,JobFamily,JobFamilyDescription,JobClass,JobClassDescription,PayGrade,EducationLevel,Experience,OrgImpact,ProblemSolving,Supervision,ContactLevel,FinancialBudget,PG
0,1,1,Accounting And Finance,1,Accountant I,5,3,1,3,3,4,3,5,PG05
1,2,1,Accounting And Finance,2,Accountant II,6,4,1,5,4,5,7,7,PG06
2,3,1,Accounting And Finance,3,Accountant III,8,4,2,6,5,6,7,10,PG08
3,4,1,Accounting And Finance,4,Accountant IV,10,5,5,6,6,7,8,11,PG10
4,5,2,Administrative Support,5,Admin Support I,1,1,0,1,1,1,1,1,PG01


Let's remove unnecessary Features

In [5]:
data = data.drop(['ID', 'JobFamily', 'JobClass', 'JobClassDescription', 'PG'], axis =1)

In [6]:
data.columns

Index(['JobFamilyDescription', 'PayGrade', 'EducationLevel', 'Experience',
       'OrgImpact', 'ProblemSolving', 'Supervision', 'ContactLevel',
       'FinancialBudget'],
      dtype='object')

We are going to separate the data.
- One part will be used to make predictions in the end,
- the other part, the most important will be used for training and testing the neural network.

In [7]:
data = shuffle(data)

In [8]:
i = 9  # Number of Features
data_to_predict = data[:i].reset_index(drop = True)

In [9]:
predict_job = data_to_predict.JobFamilyDescription 

In [10]:
predict_job = np.array(predict_job)

In [11]:
prediction = np.array(data_to_predict.drop(['JobFamilyDescription'],axis= 1))

In [12]:
prediction

array([[ 2,  1,  0,  1,  2,  1,  1,  1],
       [ 7,  4,  3,  4,  4,  4,  7,  7],
       [ 6,  2,  0,  3,  4,  1,  6,  4],
       [ 3,  1,  2,  3,  2,  1,  2,  4],
       [ 5,  3,  2,  3,  4,  5,  8,  4],
       [ 5,  4,  1,  3,  4,  4,  3,  5],
       [ 3,  1,  1,  1,  2,  1,  1,  1],
       [10,  6,  5,  6,  6,  7,  7, 11],
       [ 6,  4,  1,  4,  4,  4,  6,  2]])

In [13]:
data = data[i:].reset_index(drop = True)

In [14]:
X = data.drop(['JobFamilyDescription'], axis = 1)
X = np.array(X)
Y = data['JobFamilyDescription']

In [15]:
data.dtypes

JobFamilyDescription    object
PayGrade                 int64
EducationLevel           int64
Experience               int64
OrgImpact                int64
ProblemSolving           int64
Supervision              int64
ContactLevel             int64
FinancialBudget          int64
dtype: object

We must transform the column of classes, because we have a format `object`, and it is a multiclass situation. We must first convert the names of species into numerical values, then into vectors for the output of the neural network.

In [16]:
data['JobFamilyDescription'].value_counts()

Communications And Media    10
Administrative Support       5
Systems Analyst              5
Buildings And Facilities     5
Produce                      4
Accounting And Finance       4
Corporate Research           4
Human Resources              3
Meat Cutter                  3
Stockkeeping                 3
Buyer                        3
Cashier                      3
Baker                        2
Finance  And Accounting      2
Secretary                    1
Name: JobFamilyDescription, dtype: int64

In [17]:
# Transform name species into numerical values 
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
Y = np_utils.to_categorical(Y)
# print(Y)

# We have 15 classes : the output looks like : 
# 0,0,0,......,1 : Class 1
# 0,1,0,......,0 : Class 2
# 1,0,0,......,0 : Class 3 and so on till Class 15

### Split Training and Testing Data

In [18]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,Y,test_size = 0.1, random_state = 0)

It's time to build our neural network. The dimension in input is the number of features of the dataframe.

We are on a multiclass classification situation, so the activation function for the last most suitable layer is `softmax`, and `categorical_crossentropy` for the loss.

In [19]:
input_dim = len(data.columns) - 1

model = Sequential()
model.add(Dense(9, input_dim = input_dim , activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(15, activation = 'softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [20]:
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

In [21]:
model.fit(train_x, train_y, epochs = 300, batch_size = 1)


Epoch 1/300
51/51 [==============================] - 0s 6ms/step - loss: 3.0023 - accuracy: 0.0588
Epoch 2/300
51/51 [==============================] - 0s 1ms/step - loss: 2.7578 - accuracy: 0.0980
Epoch 3/300
51/51 [==============================] - 0s 1ms/step - loss: 2.6853 - accuracy: 0.1765
Epoch 4/300
51/51 [==============================] - 0s 1ms/step - loss: 2.6337 - accuracy: 0.1765
Epoch 5/300
51/51 [==============================] - 0s 1ms/step - loss: 2.5983 - accuracy: 0.1765
Epoch 6/300
51/51 [==============================] - 0s 1ms/step - loss: 2.5787 - accuracy: 0.1961
Epoch 7/300
51/51 [==============================] - 0s 1ms/step - loss: 2.5613 - accuracy: 0.1961
Epoch 8/300
51/51 [==============================] - 0s 1ms/step - loss: 2.5356 - accuracy: 0.1961
Epoch 9/300
51/51 [==============================] - 0s 1ms/step - loss: 2.5271 - accuracy: 0.1961
Epoch 10/300
51/51 [==============================] - 0s 1ms/step - loss: 2.4949 - accuracy: 0.1961
Epoch 11

In [22]:
scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f" % (model.metrics_names[0], scores[0]))

6/6 [==============================] - 0s 6ms/step

accuracy: 33.33%

loss: 10.84


### Predictions

It is time to make predictions with the small sample removed from the base at the beginning. To train the neural network it was necessary to convert the species into vectors.

So after the prediction it is necessary to carry out the opposite operation to recover the name of the associated species

In [23]:
predictions = model.predict_classes(prediction)
prediction_ = np.argmax(to_categorical(predictions), axis = 1)
prediction_ = encoder.inverse_transform(prediction_)

for i, j in zip(prediction_ , predict_job):
    print( "predicted category = {}, original category = {}".format(i,j))

predicted category = Communications And Media, original category = Buildings And Facilities
predicted category = Accounting And Finance, original category = Meat Cutter
predicted category = Communications And Media, original category = Baker
predicted category = Administrative Support, original category = Secretary
predicted category = Communications And Media, original category = Buildings And Facilities
predicted category = Buildings And Facilities, original category = Finance  And Accounting
predicted category = Systems Analyst, original category = Buildings And Facilities
predicted category = Systems Analyst, original category = Corporate Research
predicted category = Communications And Media, original category = Produce


/usr/local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
